COMPARAR FILTRO SEGUN BANDAS DE FRECUENCIAS MEDIANTE RESULTADOS DE CWT Y FOURIER

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pywt
from scipy.io import wavfile
from scipy.signal import decimate
from matplotlib.transforms import blended_transform_factory
from matplotlib.ticker import FixedLocator, ScalarFormatter

# Config
audio_path        = "audios/Rec096.wav"
tabla_path        = "tiempo_recortes_sincronizados.xlsx"
etiquetas_dir     = "etiquetas"
etiquetas_cwt_dir = "etiquetas_CWT"
selected_iddsi    = 4
carpeta_salida       = "figuras_cwt_filtrado/Nueva carpeta"
carpeta_figuras      = "figuras_cwt_filtrado"
carpeta_figuras_dwt  = "figuras_dwt_segmento"
carpeta_figuras_fft  = "figuras_fft_sorbos"
os.makedirs(carpeta_salida, exist_ok=True)
os.makedirs(carpeta_figuras, exist_ok=True)
os.makedirs(carpeta_figuras_dwt, exist_ok=True)
os.makedirs(carpeta_figuras_fft, exist_ok=True)

# Tabla
tabla = pd.read_excel(tabla_path)
tabla["ID_audio"] = tabla["ID_audio"].astype(str).str.strip()
audio_name   = os.path.basename(audio_path)
filas_target = tabla[(tabla["ID_audio"] == audio_name) & (tabla["IDDSI"] == selected_iddsi)]
if filas_target.empty:
    raise ValueError("No hay segmento IDDSI solicitado para ese audio")

# Audio y downsample
fs, data = wavfile.read(audio_path)
if data.ndim > 1:
    data = data[:, 0]
data = data.astype(np.float64, copy=False)
factor = 2
data = decimate(data, factor, ftype='fir', zero_phase=True)
fs = fs // factor

# Parámetros CWT
wavelet_name = "cmor2.5-1.0"
f_min, f_max = 10_000, 20
cf = pywt.central_frequency(wavelet_name)
scales = np.geomspace(cf * fs / f_max, cf * fs / f_min, num=512)

# Helpers CWT/DWT
def cargar_eventos_cwt(codigo, iddsi):
    fname = f"{codigo}.txt"
    fpath = os.path.join(etiquetas_cwt_dir, fname)
    if not os.path.exists(fpath):
        return []
    with open(fpath, encoding="utf8") as fh:
        sep_line = fh.readline()
    sep = "," if "," in sep_line else ";" if ";" in sep_line else r"\s+"
    df = pd.read_csv(fpath, sep=sep, header=None, names=["iddsi_raw", "etiqueta", "t_ini", "t_fin"])
    df["iddsi"] = pd.to_numeric(df["iddsi_raw"].astype(str).str.extract(r"(\d+)")[0], errors="coerce")
    df["t_ini"] = pd.to_numeric(df["t_ini"], errors="coerce")
    df["t_fin"] = pd.to_numeric(df["t_fin"], errors="coerce")
    df = df.dropna(subset=["iddsi", "t_ini"]).astype({"iddsi": int})
    df = df[df["iddsi"] == iddsi]
    return list(df[["t_ini", "t_fin", "etiqueta"]].itertuples(False, None))

color_map = {"E0":"orange","E1":"white","E2":"cyan","E3":"yellow","E4":"magenta","E5":"lime","E6":"red","E7":"deepskyblue"}
def _safe_name(s: str) -> str:
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in s)
def ensure_freqs_ascending(power_like, freqs):
    freqs = np.asarray(freqs)
    if freqs[0] > freqs[-1]:
        freqs = freqs[::-1]; power_like = power_like[::-1, :]
    return power_like, freqs
def edges_from_centers_log(centers):
    centers = np.asarray(centers, dtype=float)
    if centers.size < 2:
        step = 0.05 * centers[0]; return np.array([centers[0]-step, centers[0]+step])
    edges = np.empty(centers.size + 1, dtype=float)
    edges[1:-1] = np.sqrt(centers[:-1] * centers[1:])
    r0 = centers[1] / centers[0]; r1 = centers[-1] / centers[-2]
    edges[0] = centers[0] / np.sqrt(r0); edges[-1] = centers[-1] * np.sqrt(r1)
    return edges
def compress_time_for_plot(power_like, x0, x1, max_cols=3000, mode="max"):
    n_rows, n_cols = power_like.shape
    if n_cols <= max_cols:
        x_edges = np.linspace(x0, x1, n_cols + 1); return power_like, x_edges
    step = int(np.ceil(n_cols / max_cols))
    new_cols = int(np.ceil(n_cols / step))
    pad = new_cols * step - n_cols
    P = np.concatenate([power_like, np.full((n_rows, pad), np.nan, dtype=power_like.dtype)], axis=1) if pad>0 else power_like
    P = P.reshape(n_rows, new_cols, step)
    P_red = np.nanmax(P, axis=2) if mode=="max" else np.nanmean(P, axis=2)
    x_edges = np.linspace(x0, x1, new_cols + 1)
    return P_red, x_edges
def plot_spectrogram_for_view(M_plot, freqs, x0, x1, title, save_path, eventos=None, max_cols_plot=3000):
    M_plot, freqs = ensure_freqs_ascending(M_plot, freqs)
    M_plot = M_plot.astype(np.float32, copy=False)
    f_edges = edges_from_centers_log(freqs)
    Mp, x_edges = compress_time_for_plot(M_plot, x0, x1, max_cols=max_cols_plot, mode="max")
    yticks_all = [34.9,65.1,121.5,226.7,423.0,789.3,1473.1,2749.0,5129.7,9572.4]
    fmin, fmax = float(f_edges[0]), float(f_edges[-1])
    yticks = [y for y in yticks_all if (fmin <= y <= fmax)]
    plt.figure(figsize=(10, 4))
    ax = plt.gca()
    ax.pcolormesh(x_edges, f_edges, Mp, shading="auto", cmap="jet", vmin=0, vmax=1)
    ax.set_yscale("log")
    if yticks:
        ax.set_ylim(yticks[0], yticks[-1]); ax.yaxis.set_major_locator(FixedLocator(yticks))
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.ticklabel_format(axis="y", style="plain")
    plt.title(title); plt.xlabel("Tiempo (s)"); plt.ylabel("Frecuencia (Hz)")
    if eventos:
        trans = blended_transform_factory(ax.transData, ax.transAxes); y_rel = 0.97
        palette = plt.rcParams["axes.prop_cycle"].by_key()["color"]
        for idx, (t_ini, t_fin, lbl) in enumerate(eventos):
            color = color_map.get(lbl, palette[idx % len(palette)])
            ax.axvline(x=t_ini, ymin=0, ymax=1, color=color, linestyle="--", linewidth=0.9)
            ax.text(t_ini, y_rel, lbl, transform=trans, color=color,
                    ha="center", va="top", fontsize=8, fontweight="bold")
            if not pd.isna(t_fin):
                ax.axvline(x=t_fin, ymin=0, ymax=1, color=color, linestyle="--", linewidth=0.9)
    plt.tight_layout(); plt.savefig(save_path, dpi=160, bbox_inches="tight"); plt.close()

def reconstruct_approx_only(coeffs, wavelet):
    approx_coeffs = [coeffs[0]] + [np.zeros_like(c) for c in coeffs[1:]]
    return pywt.waverec(approx_coeffs, wavelet)
def reconstruct_subset(coeffs, wavelet, include_approx=True, include_details=None):
    J = len(coeffs) - 1
    include_details = include_details or []
    sel = [np.zeros_like(c) for c in coeffs]
    if include_approx:
        sel[0] = coeffs[0]
    for lvl in include_details:
        idx = J - lvl + 1
        sel[idx] = coeffs[idx]
    return pywt.waverec(sel, wavelet)
def plot_residuals_time(signals, fs, labels, title, save_path):
    t = np.arange(signals[0].size) / fs
    n = len(signals)
    fig, axes = plt.subplots(n, 1, figsize=(10, 2.2*n), sharex=True)
    if n == 1: axes = [axes]
    for ax, sig, lbl in zip(axes, signals, labels):
        color = "tab:orange" if "A10+D10+D9+D8+D1" in lbl else None
        ax.plot(t, sig, linewidth=0.8, color=color)
        ax.set_ylabel(lbl)
    axes[-1].set_xlabel("Tiempo (s)")
    fig.suptitle(title)
    fig.tight_layout()
    fig.savefig(save_path, dpi=160, bbox_inches="tight")
    plt.close(fig)

# FFT helpers
def plot_fft_sorbo(signal, fs, title, save_path, tick_step=500, fmax=3000):
    n = len(signal)
    freqs = np.fft.rfftfreq(n, d=1/fs)
    fft_vals = np.fft.rfft(signal)
    power = np.abs(fft_vals)**2
    plt.figure(figsize=(10, 3))
    plt.plot(freqs, power, color="black", linewidth=0.8)
    plt.title(title)
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Potencia (u.a.)")
    xmax = min(fmax, freqs[-1])
    plt.xlim(0, xmax)
    plt.xticks(np.arange(0, xmax + tick_step, tick_step))
    plt.tight_layout()
    plt.savefig(save_path, dpi=160, bbox_inches="tight")
    plt.close()

def plot_fft_stack(sorbos, fs, title, save_path, tick_step=500, fmax=3000):
    if not sorbos:
        return
    sorbos = sorted(sorbos, key=lambda x: x[0])
    n = len(sorbos)
    fig, axes = plt.subplots(n, 1, figsize=(10, 2.2*n), sharex=True)
    if n == 1:
        axes = [axes]
    xmax_global = None
    for (num, sig), ax in zip(sorbos, axes):
        freqs = np.fft.rfftfreq(len(sig), d=1/fs)
        power = np.abs(np.fft.rfft(sig))**2
        xmax = min(fmax, freqs[-1])
        ax.plot(freqs, power, color="black", linewidth=0.8)
        ax.set_ylabel(f"Sorbo {num}")
        ax.set_xlim(0, xmax)
        ax.tick_params(labelbottom=False)
        xmax_global = xmax
    ax_last = axes[-1]
    ax_last.tick_params(labelbottom=True, rotation=0)
    ax_last.set_xlabel("Frecuencia (Hz)")
    if xmax_global is not None:
        ax_last.xaxis.set_major_locator(mticker.MultipleLocator(500))
        ax_last.xaxis.set_minor_locator(mticker.MultipleLocator(100))
    fig.suptitle(title)
    fig.tight_layout()
    fig.savefig(save_path, dpi=160, bbox_inches="tight")
    plt.close(fig)

# Procesa señal original
for _, row in filas_target.iterrows():
    id_vol = str(row["ID_Voluntario"]).strip()
    tipo = "sujetos" if "sujeto" in id_vol.lower() else "pacientes"
    numero = ''.join(filter(str.isdigit, id_vol))
    codigo = f"S{numero}" if tipo == "sujetos" else f"P{numero}"
    t0, t1 = float(row["cut_i_audio"]), float(row["cut_f_audio"])
    i0_seg, i1_seg = int(t0 * fs), int(t1 * fs)
    seg = data[i0_seg: i1_seg]

    # CWT original
    coeffs, freqs = pywt.cwt(seg, scales, wavelet_name, sampling_period=1/fs, method="fft")
    coeffs_unnorm = coeffs * np.sqrt(scales)[:, None]; M = np.abs(coeffs_unnorm)
    eventos_cwt = cargar_eventos_cwt(codigo, selected_iddsi)
    M_plot = np.clip(M, 0, np.percentile(M, 99.0))#####################################################################################
    M_plot = (M_plot - M_plot.min()) / (M_plot.max() - M_plot.min() + 1e-12)
    titulo_seg = f"{codigo} – IDDSI {selected_iddsi} – Segmento – {audio_name}"
    nombre_seg = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_SEG_{os.path.splitext(audio_name)[0]}.png")
    ruta_fig_seg = os.path.join(carpeta_figuras, nombre_seg)
    plot_spectrogram_for_view(M_plot, freqs, t0, t1, titulo_seg, ruta_fig_seg, eventos=eventos_cwt, max_cols_plot=3000)

    # DWT 10 niveles y residuales
    dwt_wavelet = "db4"; dwt_level = 10
    coeffs_dwt = pywt.wavedec(seg, dwt_wavelet, level=dwt_level)
    approx_seg = reconstruct_approx_only(coeffs_dwt, dwt_wavelet)
    residual_A = seg - approx_seg[:len(seg)]
    combos = [
        ("A10+D10",        [10]),
        ("A10+D10+D9",     [10, 9]),
        ("A10+D10+D9+D8",  [10, 9, 8]),
        ("A10+D10+D9+D8+D7", [10, 9, 8, 7]),
    ]
    residuals = []
    labels_res = []
    for lbl, dets in combos:
        recon = reconstruct_subset(coeffs_dwt, dwt_wavelet, include_approx=True, include_details=dets)
        residuals.append(seg - recon[:len(seg)])
        labels_res.append(f"señal - ({lbl})")

    # Nuevo residual medio (quita A10, D10, D9, D8 y D1)
    recon_midband = reconstruct_subset(coeffs_dwt, dwt_wavelet, include_approx=True, include_details=[10, 9, 8, 1])
    residual_midband = seg - recon_midband[:len(seg)]
    residuals.append(residual_midband)
    labels_res.append("señal - (A10+D10+D9+D8+D1)")

    nombre_res_multi = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_SEG_RESIDUALES_MULTI_{os.path.splitext(audio_name)[0]}.png")
    ruta_res_multi = os.path.join(carpeta_figuras_dwt, nombre_res_multi)
    plot_residuals_time(residuals, fs, labels_res,
                        title=f"Residuos parciales – {codigo} IDDSI {selected_iddsi}",
                        save_path=ruta_res_multi)

    # Residual clásico
    nombre_res = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_SEG_RESIDUAL_{os.path.splitext(audio_name)[0]}.png")
    ruta_fig_res = os.path.join(carpeta_figuras_dwt, nombre_res)
    plt.figure(figsize=(10, 3))
    t = np.arange(len(seg)) / fs
    plt.plot(t, residual_A, color="tab:purple", linewidth=0.8)
    plt.title(f"Residual – {codigo} IDDSI {selected_iddsi} (señal - A{dwt_level})")
    plt.xlabel("Tiempo (s)"); plt.ylabel("Residual")
    plt.tight_layout(); plt.savefig(ruta_fig_res, dpi=160, bbox_inches="tight"); plt.close()

    # Prepara residuales de segmento para FFT por sorbo
    residual_maps = {
        "A10": residual_A,
        "A10+D10": residuals[0],
        "A10+D10+D9": residuals[1],
        "A10+D10+D9+D8": residuals[2],
        "A10+D10+D9+D8+D7": residuals[3],
        "A10+D10+D9+D8+D1": residual_midband,
    }

    # Sorbos sobre señal original + FFT por sorbo + FFT residuales apiladas
    sorbo_path = os.path.join(etiquetas_dir, f"IDDSI{selected_iddsi}_{codigo}.txt")
    if not os.path.exists(sorbo_path):
        print(f"[WARN] No hay etiquetas de sorbos: {sorbo_path}"); continue
    sorbos_df = pd.read_csv(sorbo_path, sep=r"\s+")
    sorbos_para_stack_original = []
    sorbos_para_stack_residual = {k: [] for k in residual_maps.keys()}

    for _, s in sorbos_df.iterrows():
        num, i_deg, f_deg = int(s["sorbo"]), float(s["i_deg"]), float(s["f_deg"])
        i0, i1 = int(i_deg * fs), int(f_deg * fs)

        # CWT sorbo original
        M_sorb = M[:, i0:i1]; M_plot_sorb = M_plot[:, i0:i1]
        eventos_sorb = [(ti, tf, lbl) for ti, tf, lbl in eventos_cwt if (i_deg <= ti <= f_deg)]
        titulo_sorbo = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – {audio_name}"
        nombre_sorbo = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_{os.path.splitext(audio_name)[0]}.png")
        ruta_fig_sorbo = os.path.join(carpeta_figuras, nombre_sorbo)
        plot_spectrogram_for_view(M_plot_sorb, freqs, i_deg, f_deg, titulo_sorbo, ruta_fig_sorbo, eventos=eventos_sorb, max_cols_plot=3000)

        # FFT individual sorbo original (0–3000 Hz)
        sorbo_sig = seg[i0:i1]
        titulo_fft = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – FFT Original – {audio_name}"
        nombre_fft = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_FFT_{os.path.splitext(audio_name)[0]}.png")
        ruta_fig_fft = os.path.join(carpeta_figuras_fft, nombre_fft)
        plot_fft_sorbo(sorbo_sig, fs, titulo_fft, ruta_fig_fft, tick_step=500, fmax=3000)

        sorbos_para_stack_original.append((num, sorbo_sig))

        # FFT por sorbo de cada residual (0–3000 Hz)
        for lbl, resid_sig in residual_maps.items():
            sorbos_para_stack_residual[lbl].append((num, resid_sig[i0:i1]))

        # Guardado NPZ
        carpeta_guardado = os.path.join(carpeta_salida, tipo, f"IDDSI{selected_iddsi}")
        os.makedirs(carpeta_guardado, exist_ok=True)
        nombre_archivo = f"{codigo}S{num}.npz"
        ruta_salida = os.path.join(carpeta_guardado, nombre_archivo)
        np.savez_compressed(ruta_salida, cwt_mag=M_sorb, freqs=freqs, sujeto=codigo,
                            iddsi=selected_iddsi, sorbo=num, fs=fs, t_ini=i_deg, t_fin=f_deg,
                            downsample_factor=factor, wavelet_name=wavelet_name)

    # Figuras apiladas FFT (original, 0–3000 Hz)
    if sorbos_para_stack_original:
        titulo_stack_zoom = f"{codigo} – IDDSI {selected_iddsi} – FFT sorbos (0–3000 Hz) – {audio_name}"
        nombre_stack_zoom = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_FFT_STACK_ZOOM_{os.path.splitext(audio_name)[0]}.png")
        ruta_stack_zoom = os.path.join(carpeta_figuras_fft, nombre_stack_zoom)
        plot_fft_stack(sorbos_para_stack_original, fs, titulo_stack_zoom, ruta_stack_zoom, tick_step=500, fmax=3000)

    # Figuras apiladas FFT para cada residual (0–3000 Hz)
    for lbl, lista_sorbos in sorbos_para_stack_residual.items():
        if not lista_sorbos:
            continue
        titulo_res = f"{codigo} – IDDSI {selected_iddsi} – FFT sorbos Residual señal - {lbl} (0–3000 Hz) – {audio_name}"
        nombre_res = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_FFT_STACK_{lbl}_{os.path.splitext(audio_name)[0]}.png")
        ruta_res = os.path.join(carpeta_figuras_fft, nombre_res)
        plot_fft_stack(lista_sorbos, fs, titulo_res, ruta_res, tick_step=500, fmax=3000)

print(f"Figuras CWT en: {os.path.abspath(carpeta_figuras)}")
print(f"Figuras DWT/residuales en: {os.path.abspath(carpeta_figuras_dwt)}")
print(f"Figuras FFT sorbos en: {os.path.abspath(carpeta_figuras_fft)}")
print(f"NPZ guardados en: {os.path.abspath(carpeta_salida)}")


Figuras CWT en: d:\Tesis\python\figuras_cwt_filtrado
Figuras DWT/residuales en: d:\Tesis\python\figuras_dwt_segmento
Figuras FFT sorbos en: d:\Tesis\python\figuras_fft_sorbos
NPZ guardados en: d:\Tesis\python\figuras_cwt_filtrado\Nueva carpeta


In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pywt
from scipy.io import wavfile
from scipy.signal import decimate
from matplotlib.transforms import blended_transform_factory
from matplotlib.ticker import FixedLocator, ScalarFormatter

# Config
audio_path        = "audios/Rec096.wav"
tabla_path        = "tiempo_recortes_sincronizados.xlsx"
etiquetas_dir     = "etiquetas"
etiquetas_cwt_dir = "etiquetas_CWT"
selected_iddsi    = 4
carpeta_figuras_cwt_res = "figuras_cwt_residual"
carpeta_powerdata = "Powerdata_filtrado"
os.makedirs(carpeta_figuras_cwt_res, exist_ok=True)
os.makedirs(carpeta_powerdata, exist_ok=True)

# Tabla
tabla = pd.read_excel(tabla_path)
tabla["ID_audio"] = tabla["ID_audio"].astype(str).str.strip()
audio_name   = os.path.basename(audio_path)
filas_target = tabla[(tabla["ID_audio"] == audio_name) & (tabla["IDDSI"] == selected_iddsi)]
if filas_target.empty:
    raise ValueError("No hay segmento IDDSI solicitado para ese audio")

# Audio y downsample en float32
fs, data = wavfile.read(audio_path)
if data.ndim > 1:
    data = data[:, 0]
data = data.astype(np.float32, copy=False)
factor = 2
data = decimate(data, factor, ftype='fir', zero_phase=True).astype(np.float32, copy=False)
fs = fs // factor

# Parámetros CWT en float32
wavelet_name = "cmor2.5-1.0"
f_min, f_max = 10_000, 20
cf = pywt.central_frequency(wavelet_name)
scales = np.geomspace(cf * fs / f_max, cf * fs / f_min, num=512, dtype=np.float32)

# Helpers
color_map = {"E0":"orange","E1":"white","E2":"cyan","E3":"yellow","E4":"magenta","E5":"lime","E6":"red","E7":"deepskyblue"}
def _safe_name(s: str) -> str:
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in s)
def cargar_eventos_cwt(codigo, iddsi):
    fname = f"{codigo}.txt"
    fpath = os.path.join(etiquetas_cwt_dir, fname)
    if not os.path.exists(fpath):
        return []
    with open(fpath, encoding="utf8") as fh:
        sep_line = fh.readline()
    sep = "," if "," in sep_line else ";" if ";" in sep_line else r"\s+"
    df = pd.read_csv(fpath, sep=sep, header=None, names=["iddsi_raw", "etiqueta", "t_ini", "t_fin"])
    df["iddsi"] = pd.to_numeric(df["iddsi_raw"].astype(str).str.extract(r"(\d+)")[0], errors="coerce")
    df["t_ini"] = pd.to_numeric(df["t_ini"], errors="coerce")
    df["t_fin"] = pd.to_numeric(df["t_fin"], errors="coerce")
    df = df.dropna(subset=["iddsi", "t_ini"]).astype({"iddsi": int})
    df = df[df["iddsi"] == iddsi]
    return list(df[["t_ini", "t_fin", "etiqueta"]].itertuples(False, None))
def ensure_freqs_ascending(power_like, freqs):
    freqs = np.asarray(freqs)
    if freqs[0] > freqs[-1]:
        freqs = freqs[::-1]; power_like = power_like[::-1, :]
    return power_like, freqs
def edges_from_centers_log(centers):
    centers = np.asarray(centers, dtype=float)
    if centers.size < 2:
        step = 0.05 * centers[0]; return np.array([centers[0]-step, centers[0]+step])
    edges = np.empty(centers.size + 1, dtype=float)
    edges[1:-1] = np.sqrt(centers[:-1] * centers[1:])
    r0 = centers[1]/centers[0]; r1 = centers[-1]/centers[-2]
    edges[0] = centers[0]/np.sqrt(r0); edges[-1] = centers[-1]*np.sqrt(r1)
    return edges
def compress_time_for_plot(power_like, x0, x1, max_cols=3000, mode="max"):
    n_rows, n_cols = power_like.shape
    if n_cols <= max_cols:
        x_edges = np.linspace(x0, x1, n_cols + 1); return power_like, x_edges
    step = int(np.ceil(n_cols / max_cols)); new_cols = int(np.ceil(n_cols / step))
    pad = new_cols * step - n_cols
    P = np.concatenate([power_like, np.full((n_rows, pad), np.nan, dtype=power_like.dtype)], axis=1) if pad>0 else power_like
    P = P.reshape(n_rows, new_cols, step)
    P_red = np.nanmax(P, axis=2) if mode=="max" else np.nanmean(P, axis=2)
    x_edges = np.linspace(x0, x1, new_cols + 1)
    return P_red, x_edges
def plot_spectrogram_for_view(M_plot, freqs, x0, x1, title, save_path, eventos=None, max_cols_plot=3000, vmin=None, vmax=None):
    M_plot, freqs = ensure_freqs_ascending(M_plot, freqs)
    M_plot = M_plot.astype(np.float32, copy=False)
    f_edges = edges_from_centers_log(freqs)
    Mp, x_edges = compress_time_for_plot(M_plot, x0, x1, max_cols=max_cols_plot, mode="max")
    plt.figure(figsize=(10, 4)); ax = plt.gca()
    ax.pcolormesh(x_edges, f_edges, Mp, shading="auto", cmap="jet", vmin=vmin, vmax=vmax)
    ax.set_yscale("log")
    yticks_all = [34.9,65.1,121.5,226.7,423.0,789.3,1473.1,2749.0,5129.7,9572.4]
    fmin, fmax = float(f_edges[0]), float(f_edges[-1])
    yticks = [y for y in yticks_all if (fmin <= y <= fmax)]
    if yticks:
        ax.set_ylim(yticks[0], yticks[-1]); ax.yaxis.set_major_locator(FixedLocator(yticks))
    ax.yaxis.set_major_formatter(ScalarFormatter()); ax.ticklabel_format(axis="y", style="plain")
    plt.title(title); plt.xlabel("Tiempo (s)"); plt.ylabel("Frecuencia (Hz)")
    if eventos:
        trans = blended_transform_factory(ax.transData, ax.transAxes); y_rel = 0.97
        palette = plt.rcParams["axes.prop_cycle"].by_key()["color"]
        for idx, (t_ini, t_fin, lbl) in enumerate(eventos):
            color = color_map.get(lbl, palette[idx % len(palette)])
            ax.axvline(x=t_ini, ymin=0, ymax=1, color=color, linestyle="--", linewidth=0.9)
            ax.text(t_ini, y_rel, lbl, transform=trans, color=color, ha="center", va="top", fontsize=8, fontweight="bold")
            if not pd.isna(t_fin):
                ax.axvline(x=t_fin, ymin=0, ymax=1, color=color, linestyle="--", linewidth=0.9)
    plt.tight_layout(); plt.savefig(save_path, dpi=160, bbox_inches="tight"); plt.close()

def reconstruct_approx_only(coeffs, wavelet):
    approx_coeffs = [coeffs[0]] + [np.zeros_like(c) for c in coeffs[1:]]
    return pywt.waverec(approx_coeffs, wavelet)
def reconstruct_subset(coeffs, wavelet, include_approx=True, include_details=None):
    J = len(coeffs) - 1
    include_details = include_details or []
    sel = [np.zeros_like(c) for c in coeffs]
    if include_approx:
        sel[0] = coeffs[0]
    for lvl in include_details:
        idx = J - lvl + 1
        sel[idx] = coeffs[idx]
    return pywt.waverec(sel, wavelet)

# Procesa residual y variantes
for _, row in filas_target.iterrows():
    id_vol = str(row["ID_Voluntario"]).strip()
    tipo = "sujetos" if "sujeto" in id_vol.lower() else "pacientes"
    numero = ''.join(filter(str.isdigit, id_vol))
    codigo = f"S{numero}" if tipo == "sujetos" else f"P{numero}"
    t0, t1 = float(row["cut_i_audio"]), float(row["cut_f_audio"])
    i0_seg, i1_seg = int(t0 * fs), int(t1 * fs)
    seg = data[i0_seg: i1_seg].astype(np.float32, copy=False)

    # CWT original para fijar escala global
    coeffs_orig, freqs_orig = pywt.cwt(seg, scales, wavelet_name, sampling_period=1/fs, method="fft")
    coeffs_orig = coeffs_orig.astype(np.complex64, copy=False)
    sqrt_scales = np.sqrt(scales, dtype=np.float32)
    M_orig = np.abs(coeffs_orig * sqrt_scales[:, None])
    vmin_global = 0.0
    vmax_global = np.percentile(M_orig, 99.0)##########################################################################################

    dwt_wavelet = "db4"; dwt_level_used = 10
    coeffs_dwt = pywt.wavedec(seg, dwt_wavelet, level=dwt_level_used)
    coeffs_dwt = [c.astype(np.float32, copy=False) for c in coeffs_dwt]

    combos = [
        ("A10",            [], True),
        ("A10+D10",        [10], True),
        ("A10+D10+D9",     [10, 9], True),
        ("A10+D10+D9+D8",  [10, 9, 8], True),
        ("A10+D10+D9+D8+D7", [10, 9, 8, 7], True),
        ("A10+D10+D9+D8+D1", [10, 9, 8, 1], True),  # FIL
    ]

    eventos_cwt = cargar_eventos_cwt(codigo, selected_iddsi)

    for lbl, dets, incA in combos:
        recon = reconstruct_subset(coeffs_dwt, dwt_wavelet, include_approx=incA, include_details=dets).astype(np.float32, copy=False)
        residual = seg - recon[:len(seg)]

        # CWT residual segmento
        coeffs_r, freqs_r = pywt.cwt(residual, scales, wavelet_name, sampling_period=1/fs, method="fft")
        coeffs_r = coeffs_r.astype(np.complex64, copy=False)
        M_res = np.abs(coeffs_r * sqrt_scales[:, None])
        M_res_plot = np.clip(M_res, vmin_global, vmax_global)

        if lbl == "A10+D10+D9+D8+D1":
            titulo_base = f"{codigo} – IDDSI {selected_iddsi} – Señal FIL"
        else:
            titulo_base = f"{codigo} – IDDSI {selected_iddsi} – Residual señal - {lbl}"
        nombre_seg_res = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_SEG_RESIDUAL_{lbl}_{os.path.splitext(audio_name)[0]}.png")
        ruta_fig_seg_res = os.path.join(carpeta_figuras_cwt_res, nombre_seg_res)
        plot_spectrogram_for_view(M_res_plot, freqs_r, t0, t1, titulo_base + f" – {audio_name}",
                                  ruta_fig_seg_res, eventos=eventos_cwt, max_cols_plot=3000,
                                  vmin=vmin_global, vmax=vmax_global)

        # Sorbos (mismas ventanas)
        sorbo_path = os.path.join(etiquetas_dir, f"IDDSI{selected_iddsi}_{codigo}.txt")
        if not os.path.exists(sorbo_path):
            print(f"[WARN] No hay etiquetas de sorbos: {sorbo_path}"); continue
        sorbos_df = pd.read_csv(sorbo_path, sep=r"\s+")
        for _, s in sorbos_df.iterrows():
            num, i_deg, f_deg = int(s["sorbo"]), float(s["i_deg"]), float(s["f_deg"])
            i0, i1 = int(i_deg * fs), int(f_deg * fs)
            M_res_sorb = M_res[:, i0:i1]
            M_res_plot_sorb = np.clip(M_res_sorb, vmin_global, vmax_global)

            eventos_sorb = [(ti, tf, lbl_ev) for ti, tf, lbl_ev in eventos_cwt if (i_deg <= ti <= f_deg)]

            if lbl == "A10+D10+D9+D8+D1":
                titulo_sorbo_res = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – Señal FIL – {audio_name}"
            else:
                titulo_sorbo_res = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} (Residual señal - {lbl}) – {audio_name}"

            nombre_sorbo_res = _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_RESIDUAL_{lbl}_{os.path.splitext(audio_name)[0]}.png")
            ruta_fig_sorbo_res = os.path.join(carpeta_figuras_cwt_res, nombre_sorbo_res)
            plot_spectrogram_for_view(M_res_plot_sorb, freqs_r, i_deg, f_deg,
                                      titulo_sorbo_res, ruta_fig_sorbo_res,
                                      eventos=eventos_sorb, max_cols_plot=3000,
                                      vmin=vmin_global, vmax=vmax_global)

            # Guardar NPZ crudo de la señal FIL
            if lbl == "A10+D10+D9+D8+D1":
                carpeta_guardado = os.path.join(carpeta_powerdata, tipo, f"IDDSI{selected_iddsi}")
                os.makedirs(carpeta_guardado, exist_ok=True)
                nombre_npz = f"{codigo}S{num}.npz"  # ej. P1S2
                ruta_npz = os.path.join(carpeta_guardado, nombre_npz)
                np.savez_compressed(
                    ruta_npz,
                    cwt_mag=M_res_sorb.astype(np.float32, copy=False),
                    freqs=freqs_r.astype(np.float32, copy=False),
                    sujeto=codigo,
                    iddsi=selected_iddsi,
                    sorbo=num,
                    fs=fs,
                    t_ini=i_deg,
                    t_fin=f_deg,
                    downsample_factor=factor,
                    wavelet_name=wavelet_name,
                    residual_label="FIL_A10_D10_D9_D8_D1"
                )

print(f"Figuras CWT residual en: {os.path.abspath(carpeta_figuras_cwt_res)}")
print(f"NPZ FIL en: {os.path.abspath(carpeta_powerdata)}")


Figuras CWT residual en: d:\Tesis\python\figuras_cwt_residual
NPZ FIL en: d:\Tesis\python\Powerdata_filtrado


GRAFICOS DWT Y FFT TESIS

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from scipy.io import wavfile
from scipy.signal import decimate


# ============================================================
# CONFIGURACIÓN
# ============================================================
audio_path = "audios/Rec065.wav"
tabla_path = "tiempo_recortes_sincronizados.xlsx"
etiquetas_dir = "etiquetas"

selected_iddsi = 2
down_factor = 2  # decimación para reducir tamaño

# carpetas de salida
out_dwt = "graficos_tesis/DWT"
out_fft = "graficos_tesis/FFT"

os.makedirs(out_dwt, exist_ok=True)
os.makedirs(out_fft, exist_ok=True)


# ============================================================
# CARGAR TABLA DE SEGMENTOS
# ============================================================
tabla = pd.read_excel(tabla_path)
tabla["ID_audio"] = tabla["ID_audio"].astype(str).str.strip()

audio_name = os.path.basename(audio_path)
fila = tabla[(tabla["ID_audio"] == audio_name) & (tabla["IDDSI"] == selected_iddsi)]

if fila.empty:
    raise ValueError("❌ No existe segmento IDDSI para este audio")

fila = fila.iloc[0]
t_ini_seg = float(fila["cut_i_audio"])
t_fin_seg = float(fila["cut_f_audio"])


# ============================================================
# CARGA AUDIO Y DECIMACIÓN
# ============================================================
fs, data = wavfile.read(audio_path)
if data.ndim > 1:
    data = data[:, 0]  # convertir a mono

data = data.astype(float)

data = decimate(data, down_factor, ftype="fir", zero_phase=True)
fs = fs // down_factor

seg = data[int(t_ini_seg * fs) : int(t_fin_seg * fs)]



# FFT — ticks cada 100 Hz y etiquetas cada 500 Hz

def plot_fft(signal, fs, title, save, fmax=None):
    N = len(signal)
    freqs = np.fft.rfftfreq(N, 1/fs)
    spec = np.abs(np.fft.rfft(signal))**2

    plt.figure(figsize=(10, 3))
    plt.plot(freqs, spec, color="black", linewidth=0.8)
    plt.title(title)
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Potencia")

    xmax = fmax if fmax is not None else freqs[-1]
    plt.xlim(0, xmax)

    minor_ticks = np.arange(0, xmax + 1, 100)
    major_ticks = np.arange(0, xmax + 1, 500)

    plt.gca().set_xticks(minor_ticks, minor=True)
    plt.xticks(major_ticks, labels=[str(int(t)) for t in major_ticks])

    plt.grid(False)
    plt.tight_layout()
    plt.savefig(save, dpi=160)
    plt.close()


# FUNCIÓN DWT
# ============================================================
def plot_dwt_decomposition(signal, wavelet, level, title, save, fs, t_ini):
    coeffs = pywt.wavedec(signal, wavelet, level=level)

    labels = ["Señal temporal"] + [f"A{level}"] + [f"D{i}" for i in range(level, 0, -1)]
    total_plots = len(coeffs) + 1
    plt.figure(figsize=(10, 2.2 * total_plots))

    # Señal temporal con tiempo absoluto
    t = t_ini + np.arange(len(signal)) / fs
    ax = plt.subplot(total_plots, 1, 1)
    ax.plot(t, signal, linewidth=0.8, color="black")
    ax.set_ylabel("Señal")
    ax.set_title(title)
    ax.set_xlabel("Tiempo (s)")

    # Componentes DWT
    for i, c in enumerate(coeffs):
        ax = plt.subplot(total_plots, 1, i + 2)
        ax.plot(c, linewidth=0.8)
        ax.set_ylabel(labels[i + 1])
        ax.set_xlabel("Índice coeficiente")

    plt.tight_layout()
    plt.savefig(save, dpi=160)
    plt.close()


# ============================================================
# CARGAR ETIQUETAS DE SORBOS
# ============================================================
codigo = str(fila["ID_Voluntario"]).strip()
tipo = "sujetos" if "sujeto" in codigo.lower() else "pacientes"
num = "".join(filter(str.isdigit, codigo))
codigo = f"S{num}" if tipo == "sujetos" else f"P{num}"

sorbos_path = os.path.join(etiquetas_dir, f"IDDSI{selected_iddsi}_{codigo}.txt")
df_sorbos = pd.read_csv(sorbos_path, sep=r"\s+")


# ============================================================
# PROCESAMIENTO POR SORBO — RESIDUAL REAL
# ============================================================
wavelet_dwt = "db4"
dwt_level = 10

# bandas que queremos conservar: D2–D7
keep_details = [2, 3, 4, 5, 6, 7]

for _, s in df_sorbos.iterrows():

    n = int(s["sorbo"])
    t0 = float(s["i_deg"])
    t1 = float(s["f_deg"])

    i0, i1 = int(t0 * fs), int(t1 * fs)
    sorbo_orig = seg[i0:i1]

    # DWT del sorbo
    coeffs = pywt.wavedec(sorbo_orig, wavelet_dwt, level=dwt_level)

    # Construcción del residual REAL removiendo A10, D10, D9, D8, D1
    filtered_coeffs = [np.zeros_like(c) for c in coeffs]

    for d in keep_details:
        idx = dwt_level - d + 1  # posición en la lista pywt
        filtered_coeffs[idx] = coeffs[idx]

    filtered_signal = pywt.waverec(filtered_coeffs, wavelet_dwt)
    residual = filtered_signal[:len(sorbo_orig)]

    # ============================================================
    # DWT ORIGINAL
    # ============================================================
    file_dwt_orig = os.path.join(out_dwt, f"{codigo}_S{n}_DWT_ORIGINAL.png")
    plot_dwt_decomposition(
        sorbo_orig, wavelet_dwt, dwt_level,
        title=f"{codigo} – Sorbo {n} – DWT señal original",
        save=file_dwt_orig,
        fs=fs,
        t_ini=t0
    )

    # ============================================================
    # DWT RESIDUAL
    # ============================================================
    file_dwt_res = os.path.join(out_dwt, f"{codigo}_S{n}_DWT_RESIDUAL.png")
    plot_dwt_decomposition(
        residual, wavelet_dwt, dwt_level,
        title=f"{codigo} – Sorbo {n} – DWT residual filtrado",
        save=file_dwt_res,
        fs=fs,
        t_ini=t0
    )

    # ============================================================
    # FFT ORIGINAL
    # ============================================================
    file_fft_orig_full = os.path.join(out_fft, f"{codigo}_S{n}_FFT_ORIGINAL_FULL.png")
    plot_fft(
        sorbo_orig, fs,
        title=f"{codigo} – Sorbo {n} – FFT completa señal original",
        save=file_fft_orig_full
    )

    file_fft_orig_zoom = os.path.join(out_fft, f"{codigo}_S{n}_FFT_ORIGINAL_3kHz.png")
    plot_fft(
        sorbo_orig, fs,
        title=f"{codigo} – Sorbo {n} – FFT señal original (0–3000 Hz)",
        save=file_fft_orig_zoom,
        fmax=3000
    )

    # ============================================================
    # FFT RESIDUAL REAL
    # ============================================================
    file_fft_res_full = os.path.join(out_fft, f"{codigo}_S{n}_FFT_RESIDUAL_FULL.png")
    plot_fft(
        residual, fs,
        title=f"{codigo} – Sorbo {n} – FFT residual filtrado",
        save=file_fft_res_full
    )

    file_fft_res_zoom = os.path.join(out_fft, f"{codigo}_S{n}_FFT_RESIDUAL_3kHz.png")
    plot_fft(
        residual, fs,
        title=f"{codigo} – Sorbo {n} – FFT residual (0–3000 Hz)",
        save=file_fft_res_zoom,
        fmax=3000
    )


print("\n======================================================")
print("✔ Listo Fernando. Se generaron TODOS los gráficos con RESIDUAL REAL.")
print("DWT guardados en:", os.path.abspath(out_dwt))
print(" FFT guardados en:", os.path.abspath(out_fft))
print("======================================================")



✔ Listo Fernando. Se generaron TODOS los gráficos con RESIDUAL REAL.
📂 DWT guardados en: d:\Tesis\python\graficos_tesis\DWT
📂 FFT guardados en: d:\Tesis\python\graficos_tesis\FFT
